# Quora Question pairs


Today it's a preparation for delivery 1.


The objective of today is to build a solution for:

https://www.kaggle.com/c/quora-question-pairs/overview

You might even make a late submission to kaggle to know how well you would have done in the competition.



In [1]:
import pandas as pd
import scipy
import sklearn
from sklearn import *
import numpy as np
import os

In [2]:
cl = sklearn.linear_model.Perceptron()

In [3]:
ls

02_basic_text_features.ipynb  quora_test_data.csv   train_models.ipynb
__pycache__/                  quora_train_data.csv  utils.py
delivery_1_RULES.pdf          test.py


In [4]:
path_data =  os.path.expanduser('~') 


# use this to train and VALIDATE your solution
train_df = pd.read_csv("./quora_train_data.csv")

# use this to provide the expected generalization results
test_df = pd.read_csv("./quora_test_data.csv")

In [5]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,346692,38482,10706,Why do I get easily bored with everything?,Why do I get bored with things so quickly and ...,1
1,327668,454117,345117,How do I study for Honeywell company recruitment?,How do I study for Honeywell company recruitme...,1
2,272993,391373,391374,Which search engine algorithm is Quora using?,Why is Quora not using reliable search engine?,0
3,54070,82673,95496,How can I smartly cut myself?,Can someone who thinks about suicide for 7 yea...,0
4,46450,38384,72436,How do I see who is viewing my Instagram videos?,Can one tell who viewed my Instagram videos?,1


In [6]:

A_df, te_df = sklearn.model_selection.train_test_split(train_df, test_size=0.05,random_state=123)

tr_df, va_df = sklearn.model_selection.train_test_split(A_df, test_size=0.05,random_state=123)
print('tr_df.shape=',tr_df.shape)
print('va_df.shape=',va_df.shape)
print('te_df.shape=',te_df.shape)

tr_df.shape= (291897, 6)
va_df.shape= (15363, 6)
te_df.shape= (16172, 6)


Let us train a first model....

In [7]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))

all_q1 = list(train_df["question1"]) + list(test_df["question1"])
all_q2 = list(train_df["question2"]) + list(test_df["question2"])
all_questions = all_q1 + all_q2

len(all_questions)

808580

ups! the fit method is not working

The following code will not work

```python
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
count_vectorizer.fit(all_questions)
```

In [8]:
#count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
#count_vectorizer.fit(all_questions)

We got an error what should we do?

```
ValueError: np.nan is an invalid document, expected byte or unicode string.
```


#####  Exercise 1:  `cast_list_as_strings`

Build a function  **`cast_list_as_strings`** that casts each element in the input list to a string.


In [9]:
import numpy as np
types_ = [type(q).__name__ for q in all_q1]
np.unique(types_)

array(['float', 'str'], dtype='<U5')

In [10]:
def cast_list_as_strings(mylist):
    """
    return a list of strings
    """
    #assert isinstance(mylist, list), f"the input mylist should be a list it is {type(mylist)}"
    mylist_of_strings = []
    for x in mylist:
        mylist_of_strings.append(str(x))

    return mylist_of_strings

In [11]:
q1_train =  cast_list_as_strings(list(train_df["question1"]))
q2_train =  cast_list_as_strings(list(train_df["question2"]))
q1_test  =  cast_list_as_strings(list(test_df["question1"]))
q2_test  =  cast_list_as_strings(list(test_df["question2"]))

In [12]:
import numpy as np
types_ = [type(q).__name__ for q in q2_train]
np.unique(types_)

array(['str'], dtype='<U3')


Use all the questions in train and test partitions to vuild a single list `all_questions` to fit the `count_vectorizer`

In [13]:
all_questions = q1_train + q2_train

In [14]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
count_vectorizer.fit(all_questions)

CountVectorizer()

#####  Exercise 2:  `get_features_from_df`

Make a function `get_features_from_df` that given a dataframe containing the format of the train data
it returns a scipy sparse matrix with the features from question 1 and question 2 

In [15]:

def get_features_from_df(df, count_vectorizer):
    """
    returns a sparse matrix containing the features build by the count vectorizer.
    Each row should contain features from question1 and question2.
    """
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
    
    ############### Begin exercise ###################
    # what is kaggle                  q1
    # What is the kaggle platform     q2
    X_q1 = count_vectorizer.transform(q1_casted)
    X_q2 = count_vectorizer.transform(q2_casted)    
    X_q1q2 = scipy.sparse.hstack((X_q1,X_q2))
    ############### End exercise ###################

    return X_q1q2

In [16]:
#get_features_from_df(train_df, count_vectorizer)

In [17]:
X_tr_q1q2 = get_features_from_df(train_df,count_vectorizer)
X_te_q1q2  = get_features_from_df(test_df, count_vectorizer)

X_tr_q1q2.shape, train_df.shape, test_df.shape, X_te_q1q2.shape

((323432, 156550), (323432, 6), (80858, 6), (80858, 156550))

Now You can use this representation `X_tr_q1q2` to fit a model

In [18]:
logistic = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                   random_state=123)
y_train = train_df["is_duplicate"].values
logistic.fit(X_tr_q1q2, y_train)

LogisticRegression(random_state=123, solver='liblinear')

In [19]:
# w_1 * x_1 + ... + w_n * x_n + w_n+1 * x_n+1 + ... + w_(n+m) * x_(n+m)
#  ---------------------------
#                              -----------------------------------------


In [20]:
def get_mistakes(clf, X_q1q2, y):

    ############### Begin exercise ###################
    predictions = clf.predict(X_q1q2)
    incorrect_predictions = predictions != y 
    incorrect_indices,  = np.where(incorrect_predictions)
    
    ############### End exercise ###################
    
    if np.sum(incorrect_predictions)==0:
        print("no mistakes in this df")
    else:
        return incorrect_indices, predictions

## Examine mistakes


#####  Exercise 3:  `get_mistakes`

Make a function `get_mistakes` that given a model `clf` a dataframe `df`, the features `X_q1q2` and the target labels `y`returns 


- incorrect_indices: coordinates where the model made a mistake
- predictions: predictions made by the model


In [21]:
y_train = train_df["is_duplicate"].values

mistake_indices, predictions = get_mistakes(logistic,
                                            X_tr_q1q2, 
                                            y_train)

In [22]:
def print_mistake_k(k, mistake_indices, predictions):
    print(train_df.iloc[mistake_indices[k]].question1)
    print(train_df.iloc[mistake_indices[k]].question2)
    print("true class:", train_df.iloc[mistake_indices[k]].is_duplicate)
    print("prediction:", predictions[mistake_indices[k]])

In [23]:
#[x for x in X if "'s" in x]
print_mistake_k(100, mistake_indices, predictions)

What is the best civil engineering company for a job?
What are the best companies for a civil engineer?
true class: 1
prediction: 0


In [24]:
print_mistake_k(12, mistake_indices, predictions)

Why is it so believed that aliens (if they exist) will be smarter than us human beings?
Is God an alien more intelligent than humans?
true class: 0
prediction: 1


In [25]:
# w1 w2 ... wn
# v1 v2 ... vm

In [26]:
# w1vec = word2vec.transform(w1)
# len(w1vec) = 256

# w2vec = word2vec.transform(w2)
# len(w2vec) = 256

# EmbA  = w1vec + ... + wnvec -> 256
# EmbB  = v1vec + ... + vnvec -> 256
# concat(256, 256)            -> 512
# word2vec feature: d(EmbA, EmbB)
# X_q1q2
# X_q1q2_fMaria_fPedro_fAdrian -> XGBoost / Logistic

# Improving feature vectors:

Build a Similarity metric between documents using the tf-idf vectors of the documents.

Build a class from scratch that can learn a tf-idf feature representation.

In [27]:
from utils import TfIdfCustomVectorizer, TfIdfEmbeddingVectorizer, preprocess
from gensim.models import Word2Vec

First we are going to train a Word2Vec model, so that it better fits our problem.

In [28]:
processed_questions = preprocess(all_questions)
processed_questions

[['why', 'do', 'get', 'easily', 'bored', 'with', 'everything'],
 ['how', 'do', 'study', 'for', 'honeywell', 'company', 'recruitment'],
 ['which', 'search', 'engine', 'algorithm', 'is', 'quora', 'using'],
 ['how', 'can', 'smartly', 'cut', 'myself'],
 ['how', 'do', 'see', 'who', 'is', 'viewing', 'my', 'instagram', 'videos'],
 ['what',
  'is',
  'the',
  'chance',
  'that',
  'damaged',
  'my',
  'brain',
  'smoking',
  'weed'],
 ['do',
  'quora',
  'users',
  'still',
  'see',
  'questions',
  'that',
  'are',
  'marked',
  'as',
  'needing',
  'improvement'],
 ['which',
  'politcal',
  'party',
  'would',
  'you',
  'vote',
  'for',
  'in',
  '2017',
  'vidhan',
  'sabha',
  'elections'],
 ['how',
  'much',
  'does',
  'it',
  'cost',
  'to',
  'get',
  'personal',
  'trainer',
  'at',
  'planet',
  'fitness'],
 ['what',
  'is',
  'the',
  'evolutionary',
  'significance',
  'of',
  'meterology',
  'and',
  'how',
  'has',
  'it',
  'changed',
  'modern',
  'society'],
 ['where', 'downl

In [29]:
w2v_model = Word2Vec(processed_questions, vector_size=256, window=5, min_count=1, workers=4)

In [33]:
w2v_model.save('word2vec_quora.model')